In [12]:
import pandas as pd
import copy
import string
import re
from multiprocessing import Pool, cpu_count
import time
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Descargar los recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
# Cargar el modelo de lenguaje en español de spaCy
nlp = spacy.load('es_core_news_sm')

stopwords_es = stopwords.words('spanish')
lemmatizer = WordNetLemmatizer()
PUNCT_TO_REMOVE = string.punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lcres\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lcres\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lcres\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lcres\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
dfOriginal = pd.read_csv('C:/Users/lcres/PycharmProjects/SELENIUM/yt_comments.csv', sep = ',',low_memory=False)

dataframe=copy.deepcopy(dfOriginal)
print(dataframe.shape)
dataframe.head(5)


(780, 3)


,title,url,comment
0,NaN,https://www.youtube.com/watch?v=Q6v2xdRMEzY&pp...,"- De que trata el juego? \n- De fútbol, que no..."
1,NaN,https://www.youtube.com/watch?v=Q6v2xdRMEzY&pp...,No hubo casi sustos porque auron se tomaba las...
2,NaN,https://www.youtube.com/watch?v=Q6v2xdRMEzY&pp...,3 minutos y más de 2 zooms? Aumento de sueldo ...
3,NaN,https://www.youtube.com/watch?v=Q6v2xdRMEzY&pp...,21:58 unboxing de estómago 🫦
4,NaN,https://www.youtube.com/watch?v=Q6v2xdRMEzY&pp...,La neta de los juegos q más disfruté con Auron...


In [14]:
dataframe=dataframe.drop(['title'], axis=1)
dataframe=dataframe.drop(['url'], axis=1)
print(dataframe.shape)
print(dataframe.head(10))

(780, 1)
                                             comment
0  - De que trata el juego? \n- De fútbol, que no...
1  No hubo casi sustos porque auron se tomaba las...
2  3 minutos y más de 2 zooms? Aumento de sueldo ...
3                       21:58 unboxing de estómago 🫦
4  La neta de los juegos q más disfruté con Auron...
5  El video está tan bien editado que no parece q...
6  me mata como el juego censura los genitales de...
7              6:27 " owww , soy yo con tu mama"  XD
8             6:27 pense que iba a decir algo bonito
9  Auron y los juegos de terror se complementan d...


In [15]:
dataframe["comment"] = dataframe["comment"].str.lower()

In [16]:
from nltk.tokenize import word_tokenize
# Función para remover puntuación
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
# Función para tokenizar el texto
def tokenize_text(text):
    # Handle non-string values
    if not isinstance(text, str):
        return []  # Or handle the non-string value as needed
    return word_tokenize(text)

# Función para limpiar el texto

def clean_text(tokens):
    text = " ".join(tokens)
    # Eliminar URLs
    text = re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)
    # Eliminar emojis (opción mejorada que no elimina tildes)
    text = re.sub(r'[^\w\s,]', '', text, flags=re.UNICODE)
    # Eliminar números
    text = re.sub(r'\d+', '', text)
    # Eliminar puntuación
    text = remove_punctuation(text)
    words = text.split()
    # Filtrar palabras vacías
    filtered_words = [word for word in words if word.lower() not in stopwords_es]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_words)
# Función para procesar un bloque del DataFrame
def process_block(block):
    print("Procesando..............")
    # Handle potential non-string values in 'comment' column
    block['comment'] = block['comment'].astype(str)  # Convert to string type
    block['comment_tokenized'] = block['comment'].apply(tokenize_text)
    print("Texto tokenizado")
    block['comment_cleaned'] = block['comment_tokenized'].apply(clean_text)
    print("Texto limpio")
    block['comment_lemmatized'] = block['comment_cleaned'].apply(lemmatize_text)
    print("Texto lematizado")
    return block

In [ ]:
# Dividir el DataFrame en bloques y procesar en paralelo
num_blocks = cpu_count()
blocks = [dataframe.iloc[i::num_blocks] for i in range(num_blocks)]
with Pool(num_blocks) as pool:
    result_blocks = pool.map(process_block, blocks)

result_df = pd.concat(result_blocks)
result_df.head(10)